In [5]:
"""
asset_info_extractor.py
Asset Information Extraction System

Features:
- Accepts JSON input (model_number, asset_classification_name required).
- Runs a web search and fetches page text (using DuckDuckGo lite, replaceable with a paid API).
- Calls OpenAI API with context for structured extraction.
- Parses and validates JSON output.
- Retries up to MAX_RETRIES and falls back if extraction incomplete.
- Logging included for debugging.

Reference: AI Engineer Assessment (user-uploaded spec).

Replace the line 39 with your OpenAI API key.
"""

from __future__ import annotations
import os
import json
import re
import time
import logging
from dataclasses import dataclass
from typing import Dict, List, Optional, Any, Tuple

import requests
from bs4 import BeautifulSoup
from openai import OpenAI

# ---------------- Configuration ----------------
MAX_RETRIES = 5
RETRY_INTERVAL_SECONDS = 1.0
USER_AGENT = "AssetInfoExtractor/1.0"
SEARCH_TIMEOUT = 8
FALLBACK_ASSET_CLASSIFICATION = "Generator Emissions/UREA/DPF Systems"

# OpenAI client
openai_client = OpenAI(api_key="Your API Key")

# ---------------- Logging ----------------
logger = logging.getLogger("asset_info_extractor")
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("[%(levelname)s] %(asctime)s - %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)
logger.setLevel(logging.INFO)

# ---------------- Data Models ----------------
@dataclass
class InputPayload:
    model_number: str
    asset_classification_name: str
    manufacturer: Optional[str] = ""
    asset_classification_guid2: Optional[str] = ""

    @classmethod
    def from_dict(cls, d: Dict[str, Any]) -> "InputPayload":
        if "model_number" not in d or "asset_classification_name" not in d:
            raise ValueError("model_number and asset_classification_name are required")
        return cls(
            model_number=str(d.get("model_number", "")).strip(),
            asset_classification_name=str(d.get("asset_classification_name", "")).strip(),
            manufacturer=str(d.get("manufacturer", "")).strip(),
            asset_classification_guid2=str(d.get("asset_classification_guid2", "")).strip(),
        )


@dataclass
class ExtractedAsset:
    asset_classification: Optional[str] = None
    manufacturer: Optional[str] = None
    model_number: Optional[str] = None
    product_line: Optional[str] = None
    summary: Optional[str] = None

    def is_complete(self) -> bool:
        return bool(self.asset_classification and self.model_number)

    def to_json(self) -> Dict[str, Any]:
        return {
            "asset_classification": self.asset_classification or "",
            "manufacturer": self.manufacturer or "",
            "model_number": self.model_number or "",
            "product_line": self.product_line or "",
            "summary": self.summary or "",
        }

# ---------------- Search Helpers ----------------
def build_search_query(payload: InputPayload) -> str:
    parts = [payload.model_number, payload.asset_classification_name]
    if payload.manufacturer:
        parts.append(payload.manufacturer)
    return " ".join([p for p in parts if p]).strip()

def duckduckgo_search(query: str, max_results: int = 5) -> List[str]:
    logger.debug("Searching: %s", query)
    headers = {"User-Agent": USER_AGENT}
    try:
        resp = requests.get("https://html.duckduckgo.com/html/",
                            params={"q": query}, headers=headers, timeout=SEARCH_TIMEOUT)
        resp.raise_for_status()
    except Exception as e:
        logger.warning("Search request failed: %s", e)
        return []

    soup = BeautifulSoup(resp.text, "html.parser")
    links = [a.get("href") for a in soup.select("a.result__a")[:max_results] if a.get("href")]
    return list(dict.fromkeys(links))  # dedupe

def fetch_page_text(url: str) -> str:
    headers = {"User-Agent": USER_AGENT}
    try:
        resp = requests.get(url, headers=headers, timeout=SEARCH_TIMEOUT)
        resp.raise_for_status()
    except Exception as e:
        logger.debug("Failed to fetch %s: %s", url, e)
        return ""
    soup = BeautifulSoup(resp.text, "html.parser")
    for tag in soup(["script", "style", "noscript", "header", "footer", "nav", "aside"]):
        tag.decompose()
    text = soup.get_text(" ", strip=True)
    return text[:6000]

# ---------------- LLM Prompt & Call ----------------
LLM_PROMPT_TEMPLATE = """
You are a data-extraction assistant.
Given input details and a text blob, output JSON with:

- asset_classification
- manufacturer
- model_number
- product_line
- summary (1–3 sentences)

Rules:
- Strict JSON only, no commentary.
- Leave missing fields as empty strings.
- Do not hallucinate facts.

INPUT:
MODEL_NUMBER: {model_number}
ASSET_CLASSIFICATION_NAME: {asset_name}
MANUFACTURER_HINT: {manufacturer_hint}

TEXT_BLOB:
{context}
"""

def call_llm(prompt: str, max_tokens: int = 300) -> str:
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=0.2
        )
        return response.choices[0].message.content
    except Exception as e:
        logger.error("OpenAI API call failed: %s", e)
        return "{}"

# ---------------- Parsing ----------------
def parse_llm_json(raw: str) -> ExtractedAsset:
    match = re.search(r"(\{.*\})", raw.strip(), flags=re.DOTALL)
    json_text = match.group(1) if match else raw.strip()
    try:
        data = json.loads(json_text)
    except Exception:
        return ExtractedAsset()

    def get_str(k: str) -> str:
        return str(data.get(k, "")).strip()

    return ExtractedAsset(
        asset_classification=get_str("asset_classification"),
        manufacturer=get_str("manufacturer"),
        model_number=get_str("model_number"),
        product_line=get_str("product_line"),
        summary=get_str("summary"),
    )

# ---------------- Orchestration ----------------
def single_extraction_attempt(payload: InputPayload) -> Tuple[ExtractedAsset, Dict[str, Any]]:
    debug = {"query": None, "urls": [], "llm_raw": None}
    query = build_search_query(payload)
    debug["query"] = query
    urls = duckduckgo_search(query)
    debug["urls"] = urls

    contexts = []
    for u in urls[:3]:
        txt = fetch_page_text(u)
        if txt:
            contexts.append(f"URL: {u}\n\n{txt}")
    context_blob = "\n\n---\n\n".join(contexts) or "No context found."

    prompt = LLM_PROMPT_TEMPLATE.format(
        model_number=payload.model_number,
        asset_name=payload.asset_classification_name,
        manufacturer_hint=payload.manufacturer or "",
        context=context_blob
    )
    raw = call_llm(prompt)
    debug["llm_raw"] = raw[:1000]
    return parse_llm_json(raw), debug

def extract_asset_info(payload_dict: Dict[str, Any]) -> Dict[str, Any]:
    payload = InputPayload.from_dict(payload_dict)
    attempt = 0
    last_debug = None

    while attempt < MAX_RETRIES:
        attempt += 1
        logger.info("Attempt %d/%d", attempt, MAX_RETRIES)
        extracted, debug = single_extraction_attempt(payload)
        last_debug = debug
        if extracted.is_complete():
            if extracted.model_number.lower() != payload.model_number.lower():
                extracted.model_number = payload.model_number
            return extracted.to_json()
        logger.warning("Attempt %d incomplete, retrying...", attempt)
        time.sleep(RETRY_INTERVAL_SECONDS * attempt)

    logger.error("All attempts failed, returning fallback.")
    fallback = ExtractedAsset(
        asset_classification=FALLBACK_ASSET_CLASSIFICATION,
        model_number=payload.model_number
    )
    return fallback.to_json()

# ---------------- Run Example ----------------
if __name__ == "__main__":
    example_input = {
        "model_number": "MRN85HD",
        "asset_classification_name": "Generator (Marine)"
    }
    result = extract_asset_info(example_input)
    print("FINAL OUTPUT:")
    print(json.dumps(result, indent=2))


[INFO] 2025-09-03 08:37:15,078 - Attempt 1/5
INFO:asset_info_extractor:Attempt 1/5
[ERROR] 2025-09-03 08:37:15,896 - OpenAI API call failed: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-1234u*******************************uvwx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
ERROR:asset_info_extractor:OpenAI API call failed: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-1234u*******************************uvwx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
[WARNING] 2025-09-03 08:37:15,898 - Attempt 1 incomplete, retrying...
[INFO] 2025-09-03 08:37:16,900 - Attempt 2/5
INFO:asset_info_extractor:Attempt 2/5
[ERROR] 2025-09-03 08:37:17,621 - OpenAI API call failed: Error code: 401 - {'error': {'message': 'Incorrect API key prov

FINAL OUTPUT:
{
  "asset_classification": "Generator Emissions/UREA/DPF Systems",
  "manufacturer": "",
  "model_number": "MRN85HD",
  "product_line": "",
  "summary": ""
}
